In [3]:
import pandas as pd
import re
import unicodedata

In [4]:
#fully clean strings
def clean_str(s: str) -> str:
    s = unicodedata.normalize('NFKC', s)
    for bad, good in {
        'Â': '', 'â€™': "'", '\u2019': "'", '\u2018': "'",
        '\u00A0': ' ', '\ufeff': '', '\u2026': '...'
    }.items():
        s = s.replace(bad, good)
    return s.strip()

raw_df = pd.read_excel("2024 ALS Totals for SWB.xlsx", sheet_name="Table")

In [5]:
#clean headers and text cells
raw_df.columns = [clean_str(str(c)) for c in raw_df.columns]
for col in raw_df.select_dtypes(include="object"):
    raw_df[col] = raw_df[col].apply(lambda x: clean_str(x) if isinstance(x, str) else x)

In [6]:
#functions
def extract_question_number(col_name: str) -> int:
    s = re.sub(r'[\u00A0\u202F]', ' ', col_name)
    matches = re.findall(r'(\d+)(?=[._\s])', s)
    for m in matches:
        num = int(m)
        if 1 <= num <= 66:
            return num
    return 0

def detect_response_type(series: pd.Series) -> str:
    if pd.api.types.is_numeric_dtype(series):
        return "Numeric"
    vals = series.dropna().astype(str).str.lower()
    if set(vals).issubset({"yes","no","true","false","0","1"}):
        return "Binary"
    if series.dtype == object and vals.str.contains(';').any():
        return "Checkbox"
    return "Open Text"

def detect_missing_notes(series: pd.Series) -> str:
    codes = {"refused","other","dont know","n/a","na"}
    vals = series.dropna().astype(str).str.lower()
    found = [c for c in codes if any(c in v for v in vals)]
    return ";".join(found) if found else ""

In [7]:
#build variable master ---
vm = pd.DataFrame({'Raw Column Name': raw_df.columns})

#extract base question number
vm['Qnum'] = vm['Raw Column Name'].apply(extract_question_number)

#unique survey questionnumber
vm['_dup_idx'] = vm.groupby('Qnum').cumcount().add(1)
def make_unique_qnum(row):
    q = row['Qnum']
    idx = row['_dup_idx']
    if q == 0:
        return ""
    return f"{q}.{idx}"
vm['Survey Question Number'] = vm.apply(make_unique_qnum, axis=1)
vm.drop(columns='_dup_idx', inplace=True)

In [8]:
#mappings for q1–q66
desc_map = {
    1: "League's greatest accomplishment in 2024",
    2: "League's focus areas in 2024 (multi-select)",
    3: "Number of volunteers in 2024",
    4: "Engagement in election-related activities",
    5: "Count of election-related activities hosted",
    6: "Volunteer hours invested in election activities",
    7: "LWVUS resources used for election activities",
    8: "Poll worker support activities",
    9: "Types of voter guides produced",
   10: "Number registered outside VOTE411",
   11: "Voter registration tactics used",
   12: "People reached with GOTV information",
   13: "Members serving as poll workers",
   14: "Early/mail/Election Day voting support",
   15: "Challenges voters faced on voting days",
   16: "Meetings with election officials",
   17: "Types of candidate debates/forums sponsored",
   18: "Count of debates/forums by format",
   19: "Debates/forums planned but not held",
   20: "Challenges in co-sponsoring debates/forums",
   21: "Count of non-election activities hosted",
   22: "Volunteer hours on non-election activities",
   23: "Communities engaged in 2024 (multi-select)",
   24: "Total voters reached overall",
   25: "Methods used to contact voters (multi-select)",
   26: "Civics education activities in schools",
   27: "Activities to help understand government",
   28: "Strategies against mis/disinformation",
   29: "Structured-dialogue topics hosted",
   30: "Voter education materials produced",
   31: "Media coverage earned by league",
   32: "Communications activities undertaken",
   33: "Social media platforms used",
   34: "Social media followers count",
   35: "Frequency of content updates",
   36: "Website features implemented",
   37: "Communications tools/resources needed",
   38: "Communications lead contact info",
   39: "Digital lead contact info",
   40: "Types of organizations partnered with",
   41: "Specific partner groups",
   42: "Legislative engagement in 2024",
   43: "2025 legislative priorities",
   44: "Actions toward UN Sustainable Development Goals",
   45: "Specific SDG actions taken",
   46: "Tools/resources needed for legislative work",
   47: "Comfort level with legislature engagement",
   48: "Engagement in federal court litigation",
   49: "Engagement in state court litigation",
   50: "Rating of legal representation/support",
   51: "Engagement in non-litigation legal advocacy",
   52: "Tools needed for accountability efforts",
   53: "Support needed for DEI lens application",
   54: "DEI policy adoption status",
   55: "Collection of membership demographic data",
   56: "Revenue stream estimates for 2024",
   57: "Other revenue streams (specify)",
   58: "Dedicated positions by role",
   59: "Expense areas needing funding",
   60: "Priority areas for growth/investment",
   61: "League's readiness for Nov 2024 election",
   62: "Support needed for Nov 2028 election",
   63: "Additional feedback Q63",
   64: "Additional feedback Q64",
   65: "Additional feedback Q65",
   66: "Additional feedback Q66"
}
cat_map = {
    1: "Descriptive",   2: "Descriptive",   3: "Descriptive",   4: "Diagnostic",
    5: "Descriptive",   6: "Diagnostic",    7: "Descriptive",   8: "Descriptive",
    9: "Descriptive",  10: "Descriptive",  11: "Diagnostic",   12: "Descriptive",
   13: "Descriptive",  14: "Diagnostic",   15: "Diagnostic",   16: "Diagnostic",
   17: "Descriptive",  18: "Descriptive",  19: "Descriptive",  20: "Diagnostic",
   21: "Descriptive",  22: "Descriptive",  23: "Descriptive",  24: "Descriptive",
   25: "Descriptive",  26: "Diagnostic",   27: "Diagnostic",   28: "Diagnostic",
   29: "Descriptive",  30: "Descriptive",  31: "Descriptive",  32: "Descriptive",
   33: "Descriptive",  34: "Descriptive",  35: "Descriptive",  36: "Descriptive",
   37: "Diagnostic",   38: "Descriptive",  39: "Descriptive",  40: "Descriptive",
   41: "Descriptive",  42: "Diagnostic",   43: "Descriptive",  44: "Descriptive",
   45: "Descriptive",  46: "Diagnostic",   47: "Diagnostic",   48: "Diagnostic",
   49: "Diagnostic",   50: "Diagnostic",   51: "Descriptive",  52: "Descriptive",
   53: "Descriptive",  54: "Descriptive",  55: "Descriptive",  56: "Descriptive",
   57: "Descriptive",  58: "Descriptive",  59: "Descriptive",  60: "Descriptive",
   61: "Diagnostic",   62: "Descriptive",  63: "Open Text",     64: "Open Text",
   65: "Open Text",    66: "Open Text"
}
short_map = {
     1: "greatest_accomplishment",   2: "focus_areas",           3: "volunteer_count",
     4: "election_engagement",       5: "activity_count",        6: "volunteer_hours",
     7: "resources_used",            8: "pollworker_support",    9: "guide_types",
    10: "non_vote411_regs",         11: "registration_tactics", 12: "gotv_reach",
    13: "pollworker_members",       14: "early_voting_support", 15: "voter_challenges",
    16: "officials_meetings",       17: "debate_types",         18: "debate_counts",
    19: "planned_not_held",         20: "debate_challenges",    21: "non_election_events",
    22: "non_elect_vol_hours",      23: "engaged_communities",  24: "total_voters",
    25: "contact_methods",          26: "civics_curriculum",    27: "govt_activities",
    28: "misinfo_strategies",       29: "dialogue_topics",      30: "education_materials",
    31: "media_coverage",           32: "comm_activities",      33: "social_platforms",
    34: "follower_count",           35: "update_frequency",     36: "website_features",
    37: "needed_comms_tools",       38: "comms_lead_info",      39: "digital_lead_info",
    40: "partner_org_types",        41: "partner_groups",       42: "legislative_work",
    43: "future_priorities",        44: "sdg_actions",          45: "sdg_specific",
    46: "legislative_tools",        47: "legislature_comfort",  48: "federal_litigation",
    49: "state_litigation",         50: "legal_support_rating", 51: "nonlitigation_advocacy",
    52: "accountability_tools",     53: "dei_support",          54: "dei_status",
    55: "demographic_data",         56: "revenue_estimates",    57: "other_revenues",
    58: "staff_vol_roles",          59: "expense_areas",        60: "investment_priorities",
    61: "election_readiness",       62: "support_for_2028",     63: "feedback_open1",
    64: "feedback_open2",           65: "feedback_open3",       66: "feedback_open4"
}

vm['Description']       = vm['Qnum'].map(desc_map).fillna("")
vm['Analytic Category'] = vm['Qnum'].map(cat_map).fillna("")
vm['Response Type']     = vm['Raw Column Name'].apply(lambda c: detect_response_type(raw_df[c]))
vm['Missing/Refused/Other Notes'] = vm['Raw Column Name'].apply(lambda c: detect_missing_notes(raw_df[c]))
vm['Missing (%)']       = vm['Raw Column Name'].apply(lambda c: f"{int(round(raw_df[c].isnull().mean()*100,0))}%")

In [9]:
#improved short name
def make_short_name(row):
    q = row['Qnum']
    raw = row['Raw Column Name']
    if 1 <= q <= 66:
        return short_map[q]
    words = re.findall(r'\w+', clean_str(raw))
    return "_".join(words[:3]).lower() or "col"

vm['Short Name'] = vm.apply(make_short_name, axis=1)

In [10]:
#uniqueness
counts = {}
unique_names = []
for nm in vm['Short Name']:
    counts[nm] = counts.get(nm, 0) + 1
    unique_names.append(f"{nm}_{counts[nm]}" if counts[nm] > 1 else nm)
vm['Short Name'] = unique_names

#PV sample
vm['Possible Values'] = vm['Raw Column Name'].apply(
    lambda c: ", ".join(map(str, raw_df[c].dropna().unique()[:10]))
)

In [11]:
#finalize
vm.drop(columns='Qnum', inplace=True)
vm = vm[[
    'Raw Column Name','Survey Question Number','Description',
    'Response Type','Analytic Category','Missing/Refused/Other Notes',
    'Missing (%)','Short Name','Possible Values'
]]
vm.to_csv("variable_master.csv", index=False)

In [12]:
#clean & export raw data
df = raw_df.copy()
df.columns = (
    df.columns.map(clean_str)
              .str.lower()
              .str.replace(" ", "_")
              .str.replace(r'[^\w\s]', '', regex=True)
)
df.rename(columns=dict(zip(
    vm['Raw Column Name'].map(lambda x: clean_str(x).lower().replace(" ", "_")),
    vm['Short Name']
)), inplace=True)
df.dropna(axis=1, how='all', inplace=True)
df.drop_duplicates(inplace=True)
if 'league_id' in df.columns:
    df.dropna(subset=['league_id'], inplace=True)
df.to_csv("lwv_cleaned_raw.csv", index=False)

print("✅ variable_master.csv & lwv_cleaned_raw.csv created.")

✅ variable_master.csv & lwv_cleaned_raw.csv created.


In [13]:
# clean and normalize strings
def clean_col_name(s: str) -> str:
    s = unicodedata.normalize("NFKC", str(s))
    s = s.replace(" ", "_")
    s = re.sub(r"[^\w_]", "", s)
    return s.lower()

In [14]:
#variable master to get mapping
vm = pd.read_csv("variable_master.csv")

#from cleaned raw column name to short name
col_map = {}
for _, row in vm.iterrows():
    raw = row["Raw Column Name"]
    short = row["Short Name"]
    cleaned = clean_col_name(raw)
    col_map[cleaned] = short


df = pd.read_csv("lwv_cleaned_raw.csv")

In [15]:
#rename
df = df.rename(columns={old: col_map.get(old, old) for old in df.columns})


In [16]:
#verify
renamed = [c for c in df.columns if c in col_map.values()]
print(f"Renamed {len(renamed)} columns to their short names.")

#save
df.to_csv("lwv_cleaned_shortnames.csv", index=False)
print("✅ Saved lwv_cleaned_shortnames.csv with columns renamed to short names.")


Renamed 241 columns to their short names.
✅ Saved lwv_cleaned_shortnames.csv with columns renamed to short names.
